In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Colab Setup

In [3]:
import os
# TODO: Fill in the path where you download the Assignment folder into
ROOT_PATH = "/content/drive/MyDrive/EPFL/Modern_NLP_2023/project-m3-ikun" # Replace with your directory to A3 folder
print(os.listdir(ROOT_PATH))

['model.py', '.gitignore', 'alpaca-bitcoin-sentiment-dataset.json', 'temp.ipynb', 'prompts.json', 'README.md', 'requirements.txt', 'finetune_test_llama.ipynb', 'lora-alpaca', 'templates', '.git', 'dataset', 'src', 'notebook', 'artifact', '__pycache__', 'reward-model-deberta-v3-base_collected_demo_question_answering_0.7_2e-05', 'reward-model-deberta-v3-base_collected_demo_question_answering_0.7_2e-05_peft_last_checkpoint_epoch_1']


In [4]:
%cd /content/drive/MyDrive/EPFL/Modern_NLP_2023/project-m3-ikun

/content/drive/MyDrive/EPFL/Modern_NLP_2023/project-m3-ikun


In [5]:
import sys
sys.path.append(ROOT_PATH)

### Ethical Data Generation

In [6]:
import pandas as pd

In [7]:
ethical_df = pd.read_csv('dataset/MIC.csv')

In [22]:
ethical_df.head()

,chatbot,Q,A,rot,moral,moral-vector,A_agrees,rot-agree,violation-severity,worker_answer,comments,QA,split,chosen,rejected
0,blenderbot,"Human: Am I a bad BF, weird, or just going ins...",Assistant: I don't think you're a bad bf or we...,Assistant: It's important to communicate hones...,loyalty,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",2,4.0,2,I think only your significant other can really...,NaN,"Q: Am I a bad BF, weird, or just going insane?...",train,"Human: Am I a bad BF, weird, or just going ins...","Human: Am I a bad BF, weird, or just going ins..."
1,blenderbot,"Human: in a case of double jeopardy, say you w...",Assistant: I don't think so. Attempted murder ...,Assistant: It's unlawful to engage in murder.,care|fairness,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0]",2,4.0,5,"Murder is wrong, not only by law but also reli...",NaN,"Q: in a case of double jeopardy, say you were ...",train,"Human: in a case of double jeopardy, say you w...","Human: in a case of double jeopardy, say you w..."
2,blenderbot,Human: Your house is on fire and you only have...,Assistant: It would have to be a fire extingui...,Assistant: It's important to prioritize your l...,care,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2,5.0,3,I'd just grab myself.,NaN,Q: Your house is on fire and you only have tim...,train,Human: Your house is on fire and you only have...,Human: Your house is on fire and you only have...
3,blenderbot,Human: Do you think people can be moral withou...,Assistant: I don't think so. Religion is a sou...,Assistant: It is good to have a religion.,authority|sanctity,"[0.0, 0.0, 0.0, 0.0, 1.0, 1.0]",2,4.0,4,Religion usually directs one on moral decisions.,NaN,Q: Do you think people can be moral without re...,train,Human: Do you think people can be moral withou...,Human: Do you think people can be moral withou...
4,blenderbot,Human: What is something that if you just did ...,Assistant: It would have saved me a lot of tro...,Assistant: It's good to listen to your parents...,authority,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]",2,4.0,2,If I had just gone to college like my parents ...,NaN,Q: What is something that if you just did it t...,dev,Human: What is something that if you just did ...,Human: What is something that if you just did ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113823,blenderbot,Human: What is the best way to break up with s...,Assistant: I think the best thing to do is be ...,Assistant: It is good to be honest others,care|fairness,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0]",2,5.0,2,You should let be honest and try to let them d...,NaN,Q: What is the best way to break up with someo...,train,Human: What is the best way to break up with s...,Human: What is the best way to break up with s...
113824,gpt-neo,Human: Why do we always criticize Fox News for...,"Assistant: Well, because we're just like you a...",Assistant: Expect factual information from the...,fairness|loyalty|sanctity,"[0.0, 1.0, 0.0, 1.0, 0.0, 1.0]",1,4.0,4,"It's because MSNBC mostly deals with facts, wh...",NaN,Q: Why do we always criticize Fox News for bei...,train,Human: Why do we always criticize Fox News for...,Human: Why do we always criticize Fox News for...
113825,gpt-neo,Human: How do you make long lasting friendship...,Assistant: I’ve always said that all friendshi...,"Assistant: Making new friends as an adult, can...",NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",1,3.0,3,"Making friends as an adult is hard, because yo...",NaN,Q: How do you make long lasting friendships wi...,train,Human: How do you make long lasting friendship...,Human: How do you make long lasting friendship...
113826,blenderbot,"Human: What itch would you love to scratch, bu...",Assistant: I think I would scratch the itch to...,Assistant: People should be prepared to handle...,authority,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]",0,5.0,2,I would love to date a lot of women but I know...,NaN,"Q: What itch would you love to scratch, but do...",train,"Human: What itch would you love to scratch, bu...","Human: What itch would you love 

In [8]:
ethical_df['Q']   = 'Human: ' + ethical_df['Q']
ethical_df['A']   = 'Assistant: ' + ethical_df['A']
ethical_df['rot'] = 'Assistant: ' + ethical_df['rot']

In [26]:
ethical_df['chosen'] = ethical_df['Q'] + ethical_df['rot'] + ' ' + ethical_df['worker_answer']
ethical_df['rejected'] = ethical_df['Q'] + ethical_df['A']

In [45]:
import json

# First, filter the dataframe based on the condition
# filtered_df = ethical_df[ethical_df['moral'].str.contains('care', case=False) == True][['chosen', 'rejected']].reset_index(drop=True)
filtered_df = ethical_df[['chosen', 'rejected']].reset_index(drop=True)

for idx, item in enumerate(filtered_df['rejected']):
  if type(item) == float:
    print(f"Type of item at index {idx}: {type(item)}")
    print(item)

# the dataset contain nan value, which will cause error when calling load_dataset
filtered_df = filtered_df.dropna()

# Convert the DataFrame to a dictionary
filtered_dict = filtered_df.to_dict('records')

In [46]:
# Write the dictionary to a JSON file with pretty printing
with open('dataset/ethical_subset.json', 'w') as file:
        json.dump(filtered_dict, file, indent=4, sort_keys=True)

### Fine-tune Reward Model with Ethical Data

In [ ]:
!pip install peft datasets evaluate

In [13]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from tokenizers import pre_tokenizers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from transformers.utils import PaddingStrategy

import evaluate

In [14]:
SPECIAL_TOKENS = {
    "user": "<user>",
    "assistant": "<assistant>",
    "system": "<system>",
    "bos_token": "[CLS]",
    "cls_token": "[CLS]",
    "eos_token": "[SEP]",
    "mask_token": "[MASK]",
    "pad_token": "[PAD]",
    "sep_token": "[SEP]",
    "unk_token": "[UNK]"
}

In [15]:
@dataclass
class ScriptArguments:
    """
    These arguments vary depending on how many GPUs you have, what their capacity and features are, and what size model you want to train.
    """

    resume_from_checkpoint: Optional[bool] = field(
        default=False,
        metadata={"help": "If you want to resume training where it left off."},
    )
    per_device_train_batch_size: Optional[int] = field(default=4)
    per_device_eval_batch_size: Optional[int] = field(default=1)
    gradient_accumulation_steps: Optional[int] = field(default=1)
    learning_rate: Optional[float] = field(default=2e-5)
    weight_decay: Optional[int] = field(default=0.001)
    model_name: Optional[str] = field(
        default="OpenAssistant/reward-model-deberta-v3-base", # "gpt2"
        metadata={
            "help": "The model that you want to train from the Hugging Face hub."
        },
    )
    bf16: Optional[bool] = field(
        default=False,
        metadata={
            "help": "This essentially cuts the training time in half if you want to sacrifice a little precision and have a supported GPU."
        },
    )
    num_train_epochs: Optional[int] = field(
        default=1,
        metadata={"help": "The number of training epochs for the reward model."},
    )
    train_subset: Optional[int] = field(
        default=0.7,
        metadata={"help": "The size of the subset of the training data to use"},
    )
    eval_subset: Optional[int] = field(
        default=0.3,
        metadata={"help": "The size of the subset of the eval data to use"},
    )
    gradient_checkpointing: Optional[bool] = field(
        default=False,
        metadata={"help": "Enables gradient checkpointing."},
    )
    optim: Optional[str] = field(
        default="adamw_hf",
        metadata={"help": "The optimizer to use."},
    )
    lr_scheduler_type: Optional[str] = field(
        default="linear",
        metadata={"help": "The lr scheduler"},
    )
    max_length: Optional[int] = field(default=1024)
    per_digit_tokens: Optional[bool] = field(default=False)

In [16]:
script_args = ScriptArguments(
    per_device_train_batch_size=40,
    per_device_eval_batch_size=10
)

In [47]:
dataset = load_dataset('json', data_files='dataset/ethical_subset.json')
dataset = dataset['train']
data_size = len(dataset)
if script_args.train_subset > 0:
    train_dataset = dataset.select(range(int(data_size * script_args.train_subset)))
if script_args.eval_subset > 0:
    eval_dataset = dataset.select(range(int(data_size * script_args.train_subset), data_size))
model_name_split = script_args.model_name.split("/")[-1]
# change the output name for our custom dataset later
output_name = (
    f"{model_name_split}_collected_demo_question_answering_{script_args.train_subset}_{script_args.learning_rate}"
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d71b6eab22985c80/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
print(f"Size of training dataset: {len(train_dataset)}")
print(f"Size of evaluation dataset: {len(eval_dataset)}")

Size of training dataset: 79671
Size of evaluation dataset: 34146


In [49]:
script_args.per_device_eval_batch_size

10

In [50]:
training_args = TrainingArguments(
    output_dir=output_name,
    learning_rate=script_args.learning_rate,
    per_device_train_batch_size=script_args.per_device_train_batch_size,
    per_device_eval_batch_size=script_args.per_device_eval_batch_size,
    num_train_epochs=script_args.num_train_epochs,
    weight_decay=script_args.weight_decay,
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
    gradient_checkpointing=script_args.gradient_checkpointing,
    remove_unused_columns=False,
    label_names=[],
    bf16=script_args.bf16,
    logging_strategy="steps",
    logging_steps=10,
    optim=script_args.optim,
    lr_scheduler_type=script_args.lr_scheduler_type,
)

In [51]:
# Load the value-head model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained(script_args.model_name, use_auth_token=False)
config = AutoConfig.from_pretrained(script_args.model_name)

if hasattr(config, "per_digit_tokens") and config.per_digit_tokens:
    tokenizer._tokenizer.pre_processor = pre_tokenizers.Digits(True)

# add special tokens.
if True:
    tokenizer.add_special_tokens(
        {
            "pad_token": SPECIAL_TOKENS["pad_token"],
            "eos_token": SPECIAL_TOKENS["eos_token"],
            "sep_token": SPECIAL_TOKENS["sep_token"],
        }
    )
    tokenizer.add_special_tokens(
        {"additional_special_tokens": list(SPECIAL_TOKENS.values())}
    )

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

model = AutoModelForSequenceClassification.from_pretrained(
    script_args.model_name, num_labels=1, torch_dtype=torch.bfloat16
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 296450 || all params: 184718594 || trainable%: 0.16048736273945438


In [ ]:
# Need to do this for gpt2, because it doesn't have an official pad token.
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.config.use_cache = not script_args.gradient_checkpointing
num_proc = 640  # Can adjust to be higher if you have more processors.
original_columns = train_dataset.column_names

def preprocess_function(examples):
    new_examples = {
        "input_ids_j": [],
        "attention_mask_j": [],
        "input_ids_k": [],
        "attention_mask_k": [],
    }
    for response_j, response_k in zip(examples["chosen"], examples["rejected"]):
        tokenized_j = tokenizer(response_j, truncation=True)
        tokenized_k = tokenizer(response_k, truncation=True)

        new_examples["input_ids_j"].append(tokenized_j["input_ids"])
        new_examples["attention_mask_j"].append(tokenized_j["attention_mask"])
        new_examples["input_ids_k"].append(tokenized_k["input_ids"])
        new_examples["attention_mask_k"].append(tokenized_k["attention_mask"])

    return new_examples


# preprocess the dataset and filter out QAs that are longer than script_args.max_length
train_dataset = train_dataset.map(
    preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns
)
train_dataset = train_dataset.filter(
    lambda x: len(x["input_ids_j"]) <= script_args.max_length and len(x["input_ids_k"]) <= script_args.max_length
)

eval_dataset = eval_dataset.map(preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns)
eval_dataset = eval_dataset.filter(
    lambda x: len(x["input_ids_j"]) <= script_args.max_length and len(x["input_ids_k"]) <= script_args.max_length
)


# We need to define a special data collator that batches the data in our j vs k format.
@dataclass
class RewardDataCollatorWithPadding:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        features_j = []
        features_k = []
        for feature in features:
            features_j.append(
                {
                    "input_ids": feature["input_ids_j"],
                    "attention_mask": feature["attention_mask_j"],
                }
            )
            features_k.append(
                {
                    "input_ids": feature["input_ids_k"],
                    "attention_mask": feature["attention_mask_k"],
                }
            )
        batch_j = self.tokenizer.pad(
            features_j,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch_k = self.tokenizer.pad(
            features_k,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch = {
            "input_ids_j": batch_j["input_ids"],
            "attention_mask_j": batch_j["attention_mask"],
            "input_ids_k": batch_k["input_ids"],
            "attention_mask_k": batch_k["attention_mask"],
            "return_loss": True,
        }
        return batch


# Define the metric that we'll use for validation.
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, _ = eval_pred
    # Here, predictions is rewards_j and rewards_k.
    # We want to see how much of the time rewards_j > rewards_k.
    predictions = np.argmax(predictions, axis=0)
    labels = np.zeros(predictions.shape)
    return accuracy.compute(predictions=predictions, references=labels)

In [53]:
class RewardTrainer(Trainer):
    # Define how to compute the reward loss. We use the InstructGPT pairwise logloss: https://arxiv.org/abs/2203.02155
    def compute_loss(self, model, inputs, return_outputs=False):
        rewards_j = model(input_ids=inputs["input_ids_j"], attention_mask=inputs["attention_mask_j"])[0]
        rewards_k = model(input_ids=inputs["input_ids_k"], attention_mask=inputs["attention_mask_k"])[0]
        loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
        if return_outputs:
            return loss, {"rewards_j": rewards_j, "rewards_k": rewards_k}
        return loss

trainer = RewardTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=RewardDataCollatorWithPadding(tokenizer=tokenizer, max_length=script_args.max_length),
)

# Check which device the model is on
print(f"The model is on device: {trainer.model.device}")

The model is on device: cuda:0


In [54]:
# Train the model, woohoo.
trainer.train(script_args.resume_from_checkpoint)
metrics = trainer.evaluate()
print(metrics)

print("Saving last checkpoint of the model")
model.save_pretrained(output_name + "_peft_last_checkpoint_epoch_" + str(script_args.num_train_epochs))
tokenizer.save_pretrained(output_name + "_peft_last_checkpoint_epoch_" + str(script_args.num_train_epochs))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Accuracy
200,0.350400,0.301240,0.937123
400,0.047500,0.034380,0.993147
600,0.024000,0.017680,0.996339
800,0.012300,0.013545,0.997599
1000,0.015700,0.011438,0.998038
1200,0.011100,0.010062,0.998214
1400,0.007400,0.008621,0.998653
1600,0.006600,0.007714,0.998741
1800,0.006400,0.007432,0.998741


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

{'eval_loss': 0.007307299878448248, 'eval_accuracy': 0.9987992737070228, 'eval_runtime': 247.4712, 'eval_samples_per_second': 137.98, 'eval_steps_per_second': 13.8, 'epoch': 1.0}
Saving last checkpoint of the model


('reward-model-deberta-v3-base_collected_demo_question_answering_0.7_2e-05_peft_last_checkpoint_epoch_1/tokenizer_config.json',
 'reward-model-deberta-v3-base_collected_demo_question_answering_0.7_2e-05_peft_last_checkpoint_epoch_1/special_tokens_map.json',
 'reward-model-deberta-v3-base_collected_demo_question_answering_0.7_2e-05_peft_last_checkpoint_epoch_1/tokenizer.json')